<a href="https://colab.research.google.com/github/L4riss4/Python/blob/main/TP_02_Python_para_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import datetime
import os

#Função de leitura de arquivo

def ler_arquivo_csv(nome_arquivo):
  print(f"Tabela do arquivo: {nome_arquivo} \n\n")
  df = pd.read_csv(nome_arquivo, sep=";")
  tabela = df.to_string(index=False)

  return print(f"{tabela}\n")

#Função busca do aluno pelo ra ou nome

def busca_aluno(aluno, is_ra = False):
  db_alunos = pd.read_csv('base_alunos.csv', sep=';')
  alunos_informacoes = None

  if is_ra:
    alunos_informacoes = db_alunos[db_alunos['num_ra'] == aluno]
  else:
    alunos_informacoes = db_alunos[db_alunos['nome'] == aluno]

  alunos_informacoes = alunos_informacoes.iloc[0]
  aluno_retorno = {
      "Registro" : int(alunos_informacoes['num_ra']),
      "Nome" : alunos_informacoes.get('nome', ''),
      "Curso" : alunos_informacoes.get('curso', ''),
      "Semestre" : int(alunos_informacoes.get('semestre', 0)),
      "Livros" : {
          "Livro 1" : alunos_informacoes.get('livro_1', ''),
          "Livro 2" : alunos_informacoes.get('livro_2', ''),
          "Livro 3" : alunos_informacoes.get('livro_3', '')
      }
  }
  aluno_json = json.dumps(aluno_retorno, indent=1)
  print(aluno_json)

  return aluno_retorno

#Função salvamento de objeto JSON

def salva_obj (obj_json, caminho, nome_arquivo = None):
  if nome_arquivo is None:
      agora = datetime.now()
      nome_arquivo = agora.strftime("%Y%m%d_%H%M%S") + ".json"

  caminho_arquivo = os.path.join(caminho, nome_arquivo)

  with open(caminho_arquivo, 'w', encoding='utf-8') as f:
    json.dump(obj_json, f, ensure_ascii=True, indent=2)
  return {"success": f"Arquivo salvo com sucesso em: {caminho_arquivo}"}

#Função de busca de livro por título ou ID

def busca_livro(livro, is_id= False):
  db_livros = pd.read_csv('base_livros.csv', sep=';')
  livros_informacoes = None
  resultados = []
  livros = {}

  if is_id:
    livros_informacoes = db_livros[db_livros['codigo'] == livro]
    if livros_informacoes.empty:
        return json.dumps({"error": "Livro não encontrado pelo ID."})

    for index, row in livros_informacoes.iterrows():
        livro_retorno = {
            "registro": int(row['codigo']),
            "titulo": row.get('titulo', ''),
            "autores": row.get('autores', '').split(';'),
            "volume": int(row.get('volume', 0)),
            "aluno": row.get('aluno', '')
        }
        resultados.append(livro_retorno)

    resultado_json = json.dumps(resultados, indent=2)
    print(resultado_json)
    return livro_retorno

  else:
    livros_informacoes = db_livros[db_livros['titulo'].str.contains(livro)]
    if livros_informacoes.empty:
        return json.dumps({"error": "Livro não encontrado pelo nome."})

    registros_encontrados = len(livros_informacoes)
    for index, row in livros_informacoes.iterrows():
        livro_indice = f"livro_{index + 1}"
        livros[livro_indice] = {
            "registro": int(row['codigo']),
            "titulo": row.get('titulo', ''),
            "autores": row.get('autores', '').split(';'),
            "volume": int(row.get('volume', 0)),
            "aluno": row.get('aluno', '')
        }

    livros_retorno = {
        "registros_encontrados": registros_encontrados,
        "livros": livros
    }

    resultado_json = json.dumps(livros_retorno, indent=2)
    #print(resultado_json)

    return resultado_json

In [ ]:
base_alunos = ler_arquivo_csv("base_alunos.csv")
base_livros = ler_arquivo_csv("base_livros.csv")

Tabela do arquivo: base_alunos.csv 


 num_ra             nome       curso  semestre               livro_1                         livro_2                         livro_3
      1       Joao Silva  Engenharia         1             Calculo I                        Mecanica                   Microeconomia
      2     Maria Santos     Direito         2 Introducao ao Direito          Introdução à Filosofia                Filosofia Antiga
      3  Carlos Oliveira    Medicina         3       Anatomia Humana                        Genetica Fundamentos da Biologia Celular
      4        Ana Costa Arquitetura         4 Desenho Arquitetonico           Introdução à Economia           A Origem das Espécies
      5       Pedro Lima  Computacao         5             Calculo I                      Algoritmos          Matemática para Leigos
      6    Lucas Almeida    Economia         6         Microeconomia           Introdução à Economia               O Poder do Hábito
      7 Fernanda Pereira    Bio

In [ ]:
resultado = busca_aluno(3, is_ra=True)

{
 "Registro": 3,
 "Nome": "Carlos Oliveira",
 "Curso": "Medicina",
 "Semestre": 3,
 "Livros": {
  "Livro 1": "Anatomia Humana",
  "Livro 2": "Genetica",
  "Livro 3": "Fundamentos da Biologia Celular"
 }
}


In [ ]:
busca_livro(100, is_id= True)
busca_livro('Calculo I')

'{\n  "registros_encontrados": 3,\n  "livros": {\n    "livro_1": {\n      "registro": 50,\n      "titulo": "Calculo I",\n      "autores": [\n        "James Stewart"\n      ],\n      "volume": 1,\n      "aluno": "Joao Silva"\n    },\n    "livro_22": {\n      "registro": 51,\n      "titulo": "Calculo I",\n      "autores": [\n        "James Stewart"\n      ],\n      "volume": 1,\n      "aluno": "Gabriel Mendes"\n    },\n    "livro_23": {\n      "registro": 52,\n      "titulo": "Calculo I",\n      "autores": [\n        "James Stewart"\n      ],\n      "volume": 1,\n      "aluno": "Pedro Lima"\n    }\n  }\n}'

In [ ]:
#Abrir arquivo .py no desktop
salva_obj(resultado, caminho="C:/Users/laris/Downloads", nome_arquivo="Teste.json")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/laris/Downloads/Teste.json'